In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os

os.chdir('/homes/bussotti/XFC2/code')
os.getcwd()
from explainable_fact_checking.result_presentation.notebook_utility import *
import nltk
from rank_bm25 import BM25Okapi

/home/bussotti/.conda/envs/feverous2/bin/python


# LOAD

In [2]:
from huggingface_hub import hf_hub_download

ds_path = os.path.join(xfc.experiment_definitions.C.DATASET_DIR_V2, 'AVERITEC')
preprocessed_path = os.path.join(ds_path, 'preprocessed')
os.makedirs(preprocessed_path, exist_ok=True)
files = ['train', 'dev', 'test']
files_path = {}
for f in files:
    files_path[f] = hf_hub_download(repo_id="chenxwh/AVeriTeC", filename='data/' + f + '.json', cache_dir=ds_path,
                                    revision='7f83bddcd8f16570446c51578ab3ddf6a9d350b1')

In [3]:
datasets = {}
for f in files:
    datasets[f] = json.load(open(files_path[f]))

In [4]:
e0 = datasets['train'][0]
e0

{'claim': 'Hunter Biden had no experience in Ukraine or in the energy sector when he joined the board of Burisma.',
 'required_reannotation': False,
 'label': 'Supported',
 'justification': 'No former experience stated.',
 'claim_date': '25-8-2020',
 'speaker': 'Pam Bondi',
 'original_claim_url': None,
 'fact_checking_article': 'https://web.archive.org/web/20210111003633/https://www.politifact.com/article/2020/aug/26/fact-checking-second-night-2020-rnc/',
 'reporting_source': 'Speech at The Republican National Convention',
 'location_ISO_code': 'US',
 'claim_types': ['Position Statement'],
 'fact_checking_strategies': ['Written Evidence'],
 'questions': [{'question': 'Did Hunter Biden have any experience in the energy sector at the time he joined the board of the  Burisma energy company in 2014',
   'answers': [{'answer': 'No',
     'answer_type': 'Boolean',
     'source_url': 'https://en.wikipedia.org/wiki/Hunter_Biden',
     'source_medium': 'Web text',
     'boolean_explanation': "H

# Preprocessing

In [5]:
question_answer_list = [x['questions'] for dataset in [datasets['train'], datasets['dev']] for x in dataset]
answer_distinct_fields = set()
for question_answer in question_answer_list:
    for qa in question_answer:
        for answer in qa['answers']:
            answer_distinct_fields.update(set(answer.keys()))

In [6]:
from explainable_fact_checking.dataset_preprocessing.pre import preprocess_AVERITEC


In [7]:
for prepend_question in [True, False]:
    for dataset in [datasets['train'], datasets['dev'], datasets['test']]:
        preprocess_AVERITEC(dataset, prepend_question=prepend_question)
    for f in files:
        t_name = f + ('_no_question' if prepend_question is False else '')
        with open(os.path.join(preprocessed_path, t_name + '.json'), 'w') as f_out:
            json.dump(datasets[f], f_out)

In [8]:
e0 = datasets['dev'][0]
e0

{'claim': 'In a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.',
 'required_reannotation': False,
 'label': 'Refuted',
 'justification': 'The answer and sources show that the claim was published in a fake news site so the claim is refuted.',
 'claim_date': '31-10-2020',
 'speaker': None,
 'original_claim_url': None,
 'fact_checking_article': 'https://web.archive.org/web/20201130144023/https://checkyourfact.com/2020/11/03/fact-check-sean-connery-letter-steve-jobs-apple-1998/',
 'reporting_source': 'Facebook',
 'location_ISO_code': None,
 'claim_types': ['Event/Property Claim'],
 'fact_checking_strategies': ['Written Evidence'],
 'questions': [{'question': 'Where was the claim first published',
   'answers': [{'answer': 'It was first published on Sccopertino',
     'answer_type': 'Abstractive',
     'source_url': 'https://web.archive.org/web/20201129141238/https://scoopertino.com/exposed-the-imac-disaster-that-almost-was/',
     'source_medium': 'Web text',


In [9]:
# get average number of evidence texts
sum([len(e['evidence_txt_list']) for e in datasets['train']]) / len(datasets['train'])

2.6026727509778356

In [10]:
sum([len(e['evidence_txt_list']) for e in datasets['dev']]) / len(datasets['dev'])

2.574

In [11]:
answer_distinct_fields

{'answer',
 'answer_type',
 'boolean_explanation',
 'cached_source_url',
 'source_medium',
 'source_url'}

In [12]:
datasets['test'][100]

{'claim': 'The chairman of Tirupati Bala Tirupati Balaji Temple Committee, Chandrashekhar Reddy is a Christian.',
 'claim_id': 100,
 'claim_date': '23-3-2021',
 'speaker': 'Kumar Laxman',
 'original_claim_url': 'https://www.facebook.com/kumar.laxman.75436/posts/1347004325670088',
 'reporting_source': 'Facebook',
 'location_ISO_code': 'IN',
 'id': 100,
 'evidence_txt_list': []}

In [13]:
# load datasets with questions
datasets = {}
for f in files:
    t_name = f
    with open(os.path.join(preprocessed_path, t_name + '.json'), 'r') as f_in:
        datasets[f] = json.load(f_in)

## 

In [14]:
datasets['dev'][0]


{'claim': 'In a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.',
 'required_reannotation': False,
 'label': 'Refuted',
 'justification': 'The answer and sources show that the claim was published in a fake news site so the claim is refuted.',
 'claim_date': '31-10-2020',
 'speaker': None,
 'original_claim_url': None,
 'fact_checking_article': 'https://web.archive.org/web/20201130144023/https://checkyourfact.com/2020/11/03/fact-check-sean-connery-letter-steve-jobs-apple-1998/',
 'reporting_source': 'Facebook',
 'location_ISO_code': None,
 'claim_types': ['Event/Property Claim'],
 'fact_checking_strategies': ['Written Evidence'],
 'questions': [{'question': 'Where was the claim first published',
   'answers': [{'answer': 'It was first published on Sccopertino',
     'answer_type': 'Abstractive',
     'source_url': 'https://web.archive.org/web/20201129141238/https://scoopertino.com/exposed-the-imac-disaster-that-almost-was/',
     'source_medium': 'Web text',


# Noisy evidence

In [15]:
qa_noisy_path = hf_hub_download(repo_id="chenxwh/AVeriTeC", filename='data_store/dev_top_k_qa.json', cache_dir=ds_path,
                                revision='7f83bddcd8f16570446c51578ab3ddf6a9d350b1')

In [16]:
# read json lines from the file
qa_items = []
with open(qa_noisy_path) as f:
    for line in f:
        qa_items.append(json.loads(line))

In [17]:
qa_noisy = []
for item in qa_items:
    tmp_qa = []
    for qa in item['bm25_qau']:
        tmp_qa.append(dict(question=qa[0], answers=[dict(answer=qa[1], source_url=qa[2], answer_type='Extractive')]))
    tmp_record = copy.deepcopy(item)
    tmp_record.pop('bm25_qau')
    tmp_record['questions'] = tmp_qa
    qa_noisy.append(tmp_record)

In [18]:
qa_noisy[0]

{'claim_id': 0,
 'claim': 'In a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.',
 'questions': [{'question': 'Did Sean connery send a fake letter about real Steve jobs?',
   'answers': [{'answer': 'Also, fake Sean Connery sent a letter to Real Steve Jobs.',
     'source_url': 'https://www.nbcnews.com/news/world/pre-caffeine-tech-apple-gossip-smart-pugs-flna122578',
     'answer_type': 'Extractive'}]},
  {'question': 'What is the content of this letter?',
   'answers': [{'answer': "This is a letter Sean Connery wrote didn't write in response to Steve Jobs after being asked to appear in an Apple ad.",
     'source_url': 'https://www.businessinsider.com/james-bond-sean-connery-steve-jobs-apple-letter-2011-6',
     'answer_type': 'Extractive'}]},
  {'question': 'What is the name of a fake letter that became a top twitter trend in 2020?',
   'answers': [{'answer': 'Fake Sean Connery / Steve Jobs letter becomes top Twitter trending topic',
     'source_url': 'ht

In [19]:
datasets['dev'][0]

{'claim': 'In a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.',
 'required_reannotation': False,
 'label': 'Refuted',
 'justification': 'The answer and sources show that the claim was published in a fake news site so the claim is refuted.',
 'claim_date': '31-10-2020',
 'speaker': None,
 'original_claim_url': None,
 'fact_checking_article': 'https://web.archive.org/web/20201130144023/https://checkyourfact.com/2020/11/03/fact-check-sean-connery-letter-steve-jobs-apple-1998/',
 'reporting_source': 'Facebook',
 'location_ISO_code': None,
 'claim_types': ['Event/Property Claim'],
 'fact_checking_strategies': ['Written Evidence'],
 'questions': [{'question': 'Where was the claim first published',
   'answers': [{'answer': 'It was first published on Sccopertino',
     'answer_type': 'Abstractive',
     'source_url': 'https://web.archive.org/web/20201129141238/https://scoopertino.com/exposed-the-imac-disaster-that-almost-was/',
     'source_medium': 'Web text',


In [20]:
len(qa_noisy), len(datasets['dev'])

(500, 500)

In [21]:
# for each record, compare the claim, if equal add the noisy evidence from qa_noisy but only if the source_url is not already in the gold evidence
processed_dataset = copy.deepcopy(datasets['dev'])
for i, (gold, retrieved) in enumerate(zip(processed_dataset, qa_noisy)):
    gold['claim_id'] = i
    if gold['claim'] != retrieved['claim']:
        print(f'ERROR in claim: {gold["claim"]} != {retrieved["claim"]} at index {i}')
        break
    gold['gold_evidence'] = [1] * len(list(gold['questions']))
    gold_source_urls = [a['source_url'] for q in gold['questions'] for a in q['answers']]
    for q in retrieved['questions']:
        for a in q['answers']:
            if a['source_url'] not in gold_source_urls:
                gold['questions'].append(q)
                gold['gold_evidence'].append(0)
            else:
                print(f'URL {a["source_url"]} already in gold evidence')

URL https://web.archive.org/web/20201129141238/https://scoopertino.com/exposed-the-imac-disaster-that-almost-was/ already in gold evidence
URL https://web.archive.org/web/20201129141238/https://scoopertino.com/exposed-the-imac-disaster-that-almost-was/ already in gold evidence
URL https://web.archive.org/web/20201129141238/https://scoopertino.com/exposed-the-imac-disaster-that-almost-was/ already in gold evidence
URL https://web.archive.org/web/20201129141238/https://scoopertino.com/exposed-the-imac-disaster-that-almost-was/ already in gold evidence
URL https://web.archive.org/web/20201129141238/https://scoopertino.com/exposed-the-imac-disaster-that-almost-was/ already in gold evidence
URL https://web.archive.org/web/20201129141238/https://scoopertino.com/exposed-the-imac-disaster-that-almost-was/ already in gold evidence
URL https://web.archive.org/web/20201101145631/https://nypost.com/2020/10/30/washington-post-wrongly-claims-trump-officials-criticized-billie-eilish/ already in gold 

In [22]:
t = preprocess_AVERITEC(processed_dataset)

In [23]:
pd.Series([len(x['questions']) for x in t])

0       96
1      100
2      103
3      103
4      104
      ... 
495    109
496    107
497    100
498    102
499     96
Length: 500, dtype: int64

## Filter top 5-x most similar questions

In [25]:

prompt_bm25 = BM25Okapi(tokenized_corpus)
prompt_bm25.get_scores(
                    nltk.word_tokenize(prompt_lookup_str)
                    )

NameError: name 'tokenized_corpus' is not defined